In [20]:
# Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.measure import label, regionprops
import random
import glob
import os

# Set seed for K-means clustering reproducibility
random.seed(10)

In [21]:
# K-means approach as a function
def Kmeans_process_image(image_path):
    # 1. Read image and convert to RGB
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Split into RGB channels for thresholding
    red_channel = image_rgb[:, :, 0]
    blue_channel = image_rgb[:, :, 2]
    
    # 2. Define thresholds for red and blue channels based on percentiles
    red_threshold = np.percentile(red_channel, 90)
    blue_threshold = np.percentile(blue_channel, 92)
        # Create binary masks for each color channel
    _, red_mask = cv2.threshold(red_channel, red_threshold, 255, cv2.THRESH_BINARY)
    _, blue_mask = cv2.threshold(blue_channel, blue_threshold, 255, cv2.THRESH_BINARY)
        # Combine the red and blue masks
    combined_mask = cv2.bitwise_or(red_mask, blue_mask)
        # Apply the mask to isolate high-intensity pixels in the image
    thresholded_image = cv2.bitwise_and(image_rgb, image_rgb, mask=combined_mask)

    # 3. Perform K-means clustering
        # Flatten the masked pixels (ignore dark background)
    masked_pixels = thresholded_image[combined_mask > 0]
    pixels = masked_pixels.reshape((-1, 3)).astype(np.float32)
        # Define K-means criteria and number of clusters
    k = 4  # Number of clusters
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        # Perform K-means clustering
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
        # Convert centers to uint8 for display
    centers = np.uint8(centers)
        # Reconstruct segmented data using labels
    segmented_data = centers[labels.flatten()]
        # Create an empty array to store the full-sized segmented image
    segmented_image = np.zeros_like(image_rgb)
        # Fill in only the high-intensity pixels in the original mask
    segmented_image[combined_mask > 0] = segmented_data

    # 4. Subset mask by keeping only the clusters corresponding to red and blue
        # Automate cluster characterization based on RBG values
            # instantiate cluster values
    bg_cluster = -1
    red_cluster = -1
    blue_cluster = -1
        # instantiate list of color values outside loop
    max_g_val = -1
    max_r_val = -1
    max_b_val = -1
        # 4.1: Identify BACKGROUND cluster
            # iterate over centers (RGB values)
    for idx, cntr in enumerate(centers):
        # print(idx, cntr)
            # set green value
        g_val = cntr[1]
            # compare to max green value
        if g_val > max_g_val:
                # update max green value
            max_g_val = g_val
                # set max green value to background cluster
            bg_cluster = idx
        # 4.2: Identify RED cluster
    for idx, cntr in enumerate(centers):
            # exclude cluster previously identified as bg
        if idx != bg_cluster:
            # set red value
            r_val = cntr[0]
            # compare to max red value
            if r_val > max_r_val:
                max_r_val = r_val
                red_cluster = idx
        # 4.3: Identify BLUE cluster
    for idx, cntr in enumerate(centers):
            # exclude clusters previously identified as bg & red
        if idx != bg_cluster and idx != red_cluster:
            # set red value
            b_val = cntr[2]
            # compare to max red value
            if b_val > max_b_val:
                max_b_val = b_val
                blue_cluster = idx
    # print("Background cluster is", bg_cluster)
    # print("Red cluster is", red_cluster)
    # print("Blue cluster is", blue_cluster)
        # Reshape labels to match combined_mask shape
    labels_reshaped = np.zeros(combined_mask.shape, dtype=int)
    labels_reshaped[combined_mask > 0] = labels.flatten()
        # Create a mask that includes only the red and blue clusters
    foreground_mask = np.isin(labels_reshaped, [red_cluster, blue_cluster])
        # Initialize the foreground image with black background
    foreground_final = np.zeros_like(image_rgb)
        # Apply the mask to keep only the red and blue clusters
    foreground_final[foreground_mask] = image_rgb[foreground_mask]

    # 5. Perform Canny edge detection on the final foreground
    edges = cv2.Canny(foreground_final, 100, 200, apertureSize=3, L2gradient=False)

    # 6. Find contours and filter by area
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Set minimum & maximum cell area threshold
    min_area = 100
    max_area = 1000
    cell_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area and cv2.contourArea(cnt) < max_area]
        # Initialize metrics for counting and blue:red ratio
    total_cells = len(cell_contours)
    cell_ratios = []
        # Create a copy of the original image for cell-labeling
    labeled_image = foreground_final.copy()
        # Count all filtered edges (cells) & calculate blue:red ratio
    print(f"Total cells detected: {total_cells}")

    # 7. Calculate blue:red ratios within each cell
    for i, contour in enumerate(cell_contours):
        # Create a mask for each cell
        cell_mask = np.zeros(foreground_mask.shape, dtype=np.uint8)
        cv2.drawContours(cell_mask, [contour], -1, 255, -1)  # Fill contour to create mask
        # Extract pixels in red and blue channels for this cell
        red_values = red_channel[cell_mask == 255]
        blue_values = blue_channel[cell_mask == 255]
        # Calculate blue:red ratio for this cell
        red_sum = np.sum(red_values)
        blue_sum = np.sum(blue_values)
        blue_red_ratio = blue_sum / red_sum if red_sum > 0 else 0  # Avoid division by zero
        cell_ratios.append({
            'Cell Number': i + 1,
            'Red Intensity Sum': red_sum,
            'Blue Intensity Sum': blue_sum,
            'Blue:Red Ratio': blue_red_ratio
        })
            # Label the detected cell on the image
        M = cv2.moments(contour)  # Get moments of the contour to calculate centroid
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])  # Centroid X
            cY = int(M["m01"] / M["m00"])  # Centroid Y
            cv2.putText(
                labeled_image, 
                str(i + 1),  # Label with cell number
                (cX, cY),  # Position at centroid
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.6,  # Font size
                (255, 255, 255),  # White text
                2  # Thickness
            )
        # Convert ratios to DataFrame for easy viewing
    cell_ratios_df = pd.DataFrame(cell_ratios)
        # Calculate & report image overall blue:red ratio
    total_red = np.sum(red_channel[foreground_mask])
    total_blue = np.sum(blue_channel[foreground_mask])
    overall_blue_red_ratio = total_blue / total_red if total_red > 0 else 0
        # Display results
    print("\nIndividual Cell Blue:Red Ratios")
    print(cell_ratios_df)
    print(f"\nTotal Cells: {total_cells}")
    print(f"Overall Blue:Red Ratio in Image: {overall_blue_red_ratio:.2f}")
        # Display the final cell edges
    # plt.figure(figsize=(12, 6))
    # plt.subplot(1, 3, 1)
    # plt.imshow(image_rgb)
    # plt.title('Original RGB')
    # plt.subplot(1, 3, 2)
    # plt.imshow(edges, cmap='gray')
    # plt.title('Canny Edge Detection')
    # plt.subplot(1, 3, 3)
    # plt.imshow(labeled_image)
    # plt.title('Foreground with Detected Cells')
    # plt.show()

    return cell_ratios_df, labeled_image


In [22]:
# set image path
image_path = "../images/manual_classification/raw/AS_A_I_15_4-20240720.jpg"  # WD path

# call K-means function
Kmeans_process_image(image_path)

Total cells detected: 27

Individual Cell Blue:Red Ratios
    Cell Number  Red Intensity Sum  Blue Intensity Sum  Blue:Red Ratio
0             1               4294               40405        9.409641
1             2               3574               33107        9.263290
2             3               7779               10030        1.289369
3             4               9384               93203        9.932118
4             5               6435               10711        1.664491
5             6               5639               47445        8.413726
6             7               8723               69742        7.995185
7             8              14097               32796        2.326452
8             9              11471               73406        6.399268
9            10              16967              132193        7.791183
10           11              15754              123675        7.850387
11           12               4621               11709        2.533867
12           13    

(    Cell Number  Red Intensity Sum  Blue Intensity Sum  Blue:Red Ratio
 0             1               4294               40405        9.409641
 1             2               3574               33107        9.263290
 2             3               7779               10030        1.289369
 3             4               9384               93203        9.932118
 4             5               6435               10711        1.664491
 5             6               5639               47445        8.413726
 6             7               8723               69742        7.995185
 7             8              14097               32796        2.326452
 8             9              11471               73406        6.399268
 9            10              16967              132193        7.791183
 10           11              15754              123675        7.850387
 11           12               4621               11709        2.533867
 12           13              25170              175164        6

In [24]:
# Process all images in a folder
image_folder = "../images/manual_classification/raw/"
image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))
csv_folder = "../images/Kmeans_classification/data/"
jpg_folder = "../images/Kmeans_classification/classified/"

for image_path in image_paths:
    print("")
    print(f"Processing {os.path.basename(image_path)}")
    df, image_output = Kmeans_process_image(image_path)
    # Generate basename
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    # Generate .csv output filename based on the input image name
    csv_output_path = os.path.join(csv_folder, f"{base_filename}.csv")
    image_output_path = os.path.join(jpg_folder, f"{base_filename}_labeled.jpg")

    # csv_output_filename = os.path.splitext(os.path.basename(image_path))[0] + ".csv"
    # csv_output_path = os.path.join(csv_folder, csv_output_filename)
    # Generate .jpg output filename based on the input image name
    # jpg_output_filename = os.path.splitext(os.path.basename(image_path))[0] + ".jpg"
    # jpg_output_path = os.path.join(jpg_folder, jpg_output_filename)
    
    # Save the dataframe to a CSV file
    df.to_csv(csv_output_path, index=False)
    # Save the image to a JPG file
    print(f"Saved results to {output_path}")
    # Save the labeled image to a JPG file
    cv2.imwrite(image_output_path, cv2.cvtColor(image_output, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving
    print(f"Saved labeled image to {image_output_path}")



Processing THN_STARTER_1-20240716.jpg
Total cells detected: 55

Individual Cell Blue:Red Ratios
    Cell Number  Red Intensity Sum  Blue Intensity Sum  Blue:Red Ratio
0             1               5368               21558        4.016021
1             2               3875               32667        8.430194
2             3               7418               30230        4.075222
3             4               4717               20222        4.287047
4             5              27574               29978        1.087184
5             6              28401               31703        1.116264
6             7              32272               38933        1.206402
7             8               2923               21050        7.201505
8             9              43397               54728        1.261101
9            10               6262               27204        4.344299
10           11               6170               25173        4.079903
11           12               3723               17

In [14]:
# check blue"red ratios for infection populations
# cell_ratios_df['Blue:Red Ratio'].hist()

In [ ]:
# ADD HOUGH LT TO THIS WORKFLOW